# Test Base

### Do not export

In [13]:
#| hide
from dotenv import load_dotenv
load_dotenv('/Users/shahules/Myprojects/ragas_annotator/.envrc')

True

In [14]:
#| export
import typing as t
from typing import Any, Callable, Dict, List, Optional, Union
from abc import ABC, abstractmethod
import asyncio
from dataclasses import dataclass
from pydantic import BaseModel
import openai
import instructor
from dataclasses import dataclass, field
from pydantic import BaseModel, create_model
import typing as t
import inspect

/opt/homebrew/Caskroom/miniforge/base/envs/random/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import openai
import instructor
from dataclasses import dataclass

@dataclass
class LLM:

    def __post_init__(self):
        self.aclient = instructor.from_openai(openai.AsyncOpenAI())
        self.client = instructor.from_openai(openai.OpenAI())

    
    def generate(self,prompt,response_model):
        return self.client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "user", "content": prompt},
        ],
        response_model=response_model,
    )

    async def agenerate(self,prompt,response_model):
        return await self.aclient.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "user", "content": prompt},
        ],
        response_model=response_model,
    )

/opt/homebrew/Caskroom/miniforge/base/envs/random/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## MetricResult

In [ ]:
#| export

import typing as t





class MetricResult:
    """Class to hold the result of a metric evaluation.
    
    This class behaves like its underlying result value but still provides access
    to additional metadata like reasoning.
    
    Works with:
    - DiscreteMetrics (string results)
    - NumericMetrics (float/int results)
    - RankingMetrics (list results)
    """
    
    def __init__(self, result: t.Any, reason: t.Optional[str] = None):
        self._result = result
        self.reason = reason
    
    def __repr__(self):
        return repr(self._result)
    
    # Access to underlying result
    @property
    def result(self):
        """Get the raw result value."""
        return self._result
    
    
    # String conversion - works for all types
    def __str__(self):
        return str(self._result)
    
    # Container-like behaviors for list results (RankingMetric)
    def __getitem__(self, key):
        if not hasattr(self._result, "__getitem__"):
            raise TypeError(f"{type(self._result).__name__} object is not subscriptable")
        return self._result[key]
    
    def __iter__(self):
        if not hasattr(self._result, "__iter__"):
            raise TypeError(f"{type(self._result).__name__} object is not iterable")
        return iter(self._result)
    
    def __len__(self):
        if not hasattr(self._result, "__len__"):
            raise TypeError(f"{type(self._result).__name__} has no len()")
        return len(self._result)
    
    # Numeric operations for numeric results (NumericMetric)
    def __float__(self):
        if isinstance(self._result, (int, float)):
            return float(self._result)
        raise TypeError(f"Cannot convert {type(self._result).__name__} to float")
    
    def __int__(self):
        if isinstance(self._result, (int, float)):
            return int(self._result)
        raise TypeError(f"Cannot convert {type(self._result).__name__} to int")
    
    def __add__(self, other):
        if not isinstance(self._result, (int, float)):
            raise TypeError(f"Cannot add {type(self._result).__name__} objects")
        if isinstance(other, MetricResult):
            return self._result + other._result
        return self._result + other
    
    def __radd__(self, other):
        if not isinstance(self._result, (int, float)):
            raise TypeError(f"Cannot add {type(self._result).__name__} objects")
        return other + self._result
    
    def __sub__(self, other):
        if not isinstance(self._result, (int, float)):
            raise TypeError(f"Cannot subtract {type(self._result).__name__} objects")
        if isinstance(other, MetricResult):
            return self._result - other._result
        return self._result - other
    
    def __rsub__(self, other):
        if not isinstance(self._result, (int, float)):
            raise TypeError(f"Cannot subtract {type(self._result).__name__} objects")
        return other - self._result
    
    def __mul__(self, other):
        if not isinstance(self._result, (int, float)):
            raise TypeError(f"Cannot multiply {type(self._result).__name__} objects")
        if isinstance(other, MetricResult):
            return self._result * other._result
        return self._result * other
    
    def __rmul__(self, other):
        if not isinstance(self._result, (int, float)):
            raise TypeError(f"Cannot multiply {type(self._result).__name__} objects")
        return other * self._result
    
    def __truediv__(self, other):
        if not isinstance(self._result, (int, float)):
            raise TypeError(f"Cannot divide {type(self._result).__name__} objects")
        if isinstance(other, MetricResult):
            return self._result / other._result
        return self._result / other
    
    def __rtruediv__(self, other):
        if not isinstance(self._result, (int, float)):
            raise TypeError(f"Cannot divide {type(self._result).__name__} objects")
        return other / self._result
    
    # Comparison operations - work for all types with same-type comparisons
    def __eq__(self, other):
        if isinstance(other, MetricResult):
            return self._result == other._result
        return self._result == other
    
    def __lt__(self, other):
        if isinstance(other, MetricResult):
            return self._result < other._result
        return self._result < other
    
    def __le__(self, other):
        if isinstance(other, MetricResult):
            return self._result <= other._result
        return self._result <= other
    
    def __gt__(self, other):
        if isinstance(other, MetricResult):
            return self._result > other._result
        return self._result > other
    
    def __ge__(self, other):
        if isinstance(other, MetricResult):
            return self._result >= other._result
        return self._result >= other
    
    # Method forwarding for type-specific behaviors
    def __getattr__(self, name):
        """Forward attribute access to the result object if it has that attribute.
        
        This allows calling string methods on discrete results, 
        numeric methods on numeric results, and list methods on ranking results.
        """
        if hasattr(self._result, name):
            attr = getattr(self._result, name)
            if callable(attr):
                # If it's a method, wrap it to return MetricResult when appropriate
                def wrapper(*args, **kwargs):
                    result = attr(*args, **kwargs)
                    # If the result is of the same type as self._result, wrap it
                    if isinstance(result, type(self._result)):
                        return MetricResult(result=result, reason=self.reason)
                    return result
                return wrapper
            return attr
        raise AttributeError(f"{type(self).__name__} has no attribute '{name}'")
    
    # JSON/dict serialization
    def to_dict(self):
        """Convert the result to a dictionary."""
        return {
            "result": self._result,
            "reason": self.reason
        }

### Metric

In [ ]:
#| export

from abc import ABC, abstractmethod
import asyncio
from dataclasses import dataclass, field
from pydantic import BaseModel
import typing as t
from ragas_annotator.metric import MetricResult

@dataclass
class Metric(ABC):
    """Base class for all metrics in the LLM evaluation library."""
    name: str
    prompt: str
    llm: 'LLM'  # Forward reference with quotes
    _response_models: t.Dict[bool, t.Type[BaseModel]] = field(
        default_factory=dict, init=False, repr=False
    )
    
    @abstractmethod
    def _get_response_model(self, with_reasoning: bool) -> t.Type[BaseModel]:
        """Get the appropriate response model."""
        pass

    @abstractmethod
    def _ensemble(self, results: t.List[MetricResult]) -> MetricResult:
        pass
        
    
    def score(self, reasoning: bool = True, n: int = 1, **kwargs) -> t.Any:
        responses = []
        prompt_input = self.prompt.format(**kwargs)
        for _ in range(n):
            response = self.llm.generate(prompt_input, response_model = self._get_response_model(reasoning)) 
            response = MetricResult(**response.model_dump())
            responses.append(response)
        return self._ensemble(responses)


    async def ascore(self, reasoning: bool = True, n: int = 1, **kwargs) -> MetricResult:
        responses = []  # Added missing initialization
        prompt_input = self.prompt.format(**kwargs)
        for _ in range(n):
            response = await self.llm.agenerate(prompt_input, response_model = self._get_response_model(reasoning))
            response = MetricResult(**response.model_dump())  # Fixed missing parentheses
            responses.append(response)
        return self._ensemble(responses)
        
    def batch_score(self, inputs: t.List[t.Dict[str, t.Any]], reasoning: bool = True, n: int = 1) -> t.List[t.Any]:
        return [self.score(reasoning, n, **input_dict) for input_dict in inputs]
    
    async def abatch_score(self, inputs: t.List[t.Dict[str, t.Any]], reasoning: bool = True, n: int = 1) -> t.List[MetricResult]:
        async_tasks = []
        for input_dict in inputs:
            # Add reasoning and n to the input parameters
            async_tasks.append(self.ascore(reasoning=reasoning, n=n, **input_dict))
            
        # Run all tasks concurrently and return results
        return await asyncio.gather(*async_tasks)

### DiscreteMetric 


In [ ]:
#| export
from collections import Counter

@dataclass
class DiscreteMetrics(Metric):
    values: t.List[str] = field(default_factory=lambda: ["pass", "fail"])
    
    def _get_response_model(self, with_reasoning: bool) -> t.Type[BaseModel]:
        """Get or create a response model based on reasoning parameter."""
        
        if with_reasoning in self._response_models:
            return self._response_models[with_reasoning]
        
        model_name = 'response_model'
        values = tuple(self.values)
        fields = {"result": (t.Literal[values], ...)}
        
        if with_reasoning:
            fields["reason"] = (str, ...)
        
        model = create_model(model_name, **fields)
        self._response_models[with_reasoning] = model
        return model 

    def _ensemble(self,results:t.List[MetricResult]) -> MetricResult:


        if len(results)==1:
            return results[0]
            
        candidates = [candidate.result for candidate in results]
        counter = Counter(candidates)
        max_count = max(counter.values())
        for candidate in results:
            if counter[candidate.result] == max_count:
                result = candidate.result              
                reason = candidate.reason
                break
        
        return MetricResult(result=result,reason=reason)


    

### decorator factory for discrete_metric

In [110]:
import typing as t
from typing import Any, Callable, Dict, List, Optional, Type, Union
import inspect
import asyncio
from dataclasses import dataclass
from abc import ABC

def create_metric_decorator(metric_class):
    """
    Factory function that creates decorator factories for different metric types.
    
    Args:
        metric_class: The metric class to use (DiscreteMetrics, NumericMetrics, etc.)
        
    Returns:
        A decorator factory function for the specified metric type
    """
    def decorator_factory(llm, prompt, name: t.Optional[str] = None, **metric_params):
        """
        Creates a decorator that wraps a function into a metric instance.
        
        Args:
            llm: The language model instance to use
            prompt: The prompt template
            name: Optional name for the metric (defaults to function name)
            **metric_params: Additional parameters specific to the metric type
                (values for DiscreteMetrics, range for NumericMetrics, etc.)
        
        Returns:
            A decorator function
        """
        def decorator(func):
            # Get metric name and check if function is async
            metric_name = name or func.__name__
            is_async = inspect.iscoroutinefunction(func)
            
            @dataclass
            class CustomMetric(metric_class):
                def _extract_result(self, result, reasoning: bool):
                    """Extract score and reason from the result."""
                    if isinstance(result, tuple) and len(result) == 2:
                        score, reason = result
                    else:
                        score, reason = result, None
                    
                    # Use "result" instead of "score" for the new MetricResult implementation
                    return MetricResult(result=score, reason=reason if reasoning else None)
                
                def _run_sync_in_async(self, func, *args, **kwargs):
                    """Run a synchronous function in an async context."""
                    # For sync functions, just run them normally
                    return func(*args, **kwargs)
                
                def _execute_metric(self, is_async_execution, reasoning, **kwargs):
                    """Execute the metric function with proper async handling."""
                    try:
                        if is_async:
                            # Async function implementation
                            if is_async_execution:
                                # In async context, await the function directly
                                result = func(self.llm, self.prompt, **kwargs)
                            else:
                                # In sync context, run the async function in an event loop
                                try:
                                    loop = asyncio.get_event_loop()
                                except RuntimeError:
                                    loop = asyncio.new_event_loop()
                                    asyncio.set_event_loop(loop)
                                result = loop.run_until_complete(func(self.llm, self.prompt, **kwargs))
                        else:
                            # Sync function implementation
                            result = func(self.llm, self.prompt, **kwargs)
                        
                        return self._extract_result(result, reasoning)
                    except Exception as e:
                        # Handle errors gracefully
                        error_msg = f"Error executing metric {self.name}: {str(e)}"
                        return MetricResult(result=None, reason=error_msg)
                
                def score(self, reasoning: bool = True, n: int = 1, **kwargs):
                    """Synchronous scoring method."""
                    return self._execute_metric(is_async_execution=False, reasoning=reasoning, **kwargs)
                
                async def ascore(self, reasoning: bool = True, n: int = 1, **kwargs):
                    """Asynchronous scoring method."""
                    if is_async:
                        # For async functions, await the result
                        result = await func(self.llm, self.prompt, **kwargs)
                        return self._extract_result(result, reasoning)
                    else:
                        # For sync functions, run normally
                        result = self._run_sync_in_async(func, self.llm, self.prompt, **kwargs)
                        return self._extract_result(result, reasoning)
            
            # Create the metric instance with all parameters
            metric_instance = CustomMetric(
                name=metric_name,
                prompt=prompt,
                llm=llm,
                **metric_params
            )
            
            # Preserve metadata
            metric_instance.__name__ = metric_name
            metric_instance.__doc__ = func.__doc__
            
            return metric_instance
        
        return decorator
    
    return decorator_factory

# Create specific decorator factories for each metric type
discrete_metric = create_metric_decorator(DiscreteMetrics)


### Usage pattern

In [113]:

my_metric = DiscreteMetrics(
    name='helpfulness',
    llm=LLM(),
    prompt="Evaluate if given answer is helpful\n\n{response}",
    values=["low","med","high"],
)

result = my_metric.score(response="this is my response")
result #gives "low"
result.reason #gives reasoning from llm



"The answer provided lacks specific context or detail needed to evaluate its helpfulness fully. Without more information, it's difficult to determine its applicability."

In [114]:
## score without reasoning to save reasoning tokens cost
result = my_metric.score(response="this is my response",reasoning=False,n=3)
result

'low'

In [115]:
@discrete_metric(llm=LLM(),
    prompt="Evaluate if given answer is helpful\n\n{response}",
    name='new_metric',values=["low","med","high"])
def my_metric(llm,prompt,**kwargs):

        class response_model(BaseModel):
             output: t.List[bool]
             reason: str
        
        response = llm.generate(prompt.format(**kwargs),response_model=response_model)
        total = sum(response.output)
        if total < 1:
            score = 'low'
        else:
            score = 'high'
        return score,"reason"

result = my_metric.score(response='my response') # result
result
result.reason

'reason'

## Numeric Metric

In [124]:
#| export

@dataclass
class NumericMetrics(Metric):
    range: t.Tuple[float,float]
    
    def _get_response_model(self, with_reasoning: bool) -> t.Type[BaseModel]:
        """Get or create a response model based on reasoning parameter."""
        
        if with_reasoning in self._response_models:
            return self._response_models[with_reasoning]
        
        model_name = 'response_model'
        fields = {"result": (float,...)}
        
        if with_reasoning:
            fields["reason"] = (str, ...)
        
        model = create_model(model_name, **fields)
        self._response_models[with_reasoning] = model
        return model 

    def _ensemble(self,results:t.List[MetricResult]) -> MetricResult:

        if len(results)==1:
            return results[0]
    
        candidates = [candidate.result for candidate in results]
        result = sum(candidates)/len(candidates)
        reason = results[0].reason
    
        return MetricResult(result=result,reason=reason)
    


In [129]:
my_metric = NumericMetrics(
    name='helpfulness',
    llm=LLM(),
    prompt="Evaluate if given answer is helpful\n\n{response}",
    range=(0,10),
)

result = my_metric.score(response="this is my response")
result #gives "low"
result.reason #gives reasoning from llm



'The response lacks sufficient information or context to be considered helpful. It does not address any specific question or provide any useful insights.'

In [130]:
my_metric.batch_score(inputs=[{"response":"this is my response"}])


[0.0]

### decorator factory

In [120]:
#| export
numeric_metric = create_metric_decorator(NumericMetrics)

In [132]:
@numeric_metric(llm=LLM(),
    prompt="Evaluate if given answer is helpful\n\n{response}",
    name='new_metric',range=(0,10))
def my_metric(llm,prompt,**kwargs):

        class response_model(BaseModel):
             output: int
             reason: str
        
        response = llm.generate(prompt.format(**kwargs),response_model=response_model)
        total = response.output
        if total < 1:
            score = 0
        else:
            score = 10
        return score,"reason"

result = my_metric.score(response='my response') # result
result # 10
result.reason # the reason for the answer

result1 = my_metric.score(response='my response 1') # result
result2 = my_metric.score(response='my response 2') # result

result1 + result2 # should be addable and behave like a float


20

### Ranking metric

In [145]:
t.Literal[[i for i in range(3)]]

typing.Literal[[0, 1, 2]]

In [159]:
#| export

@dataclass
class RankingMetrics(Metric):
    num_ranks: int
    
    def _get_response_model(self, with_reasoning: bool) -> t.Type[BaseModel]:
        """Get or create a response model based on reasoning parameter."""
        
        if with_reasoning in self._response_models:
            return self._response_models[with_reasoning]
        
        model_name = 'response_model'

        # Custom validator to ensure 'result' is a permutation of 0 .. num_ranks-1
        def validate_result(cls, v):
            expected = set(range(self.num_ranks))
            if set(v) != expected:
                raise ValueError(
                    f"'result' must contain exactly the numbers {sorted(expected)} without repetition."
                )
            return v

        # Define the fields dynamically
        fields = {"result": (List[int], ...)}
        if with_reasoning:
            fields["reason"] = (str, ...)
        
        # Create the dynamic model with the custom validator attached
        model = create_model(
            model_name,
            **fields,
            __validators__={
                'result_validator': validator('result', allow_reuse=True)(validate_result)
            }
        )
        self._response_models[with_reasoning] = model
        return model 

    def _ensemble(self,results:t.List[MetricResult]) -> MetricResult:

        if len(results)==1:
            return results[0]

        n_items = len(results)
        borda_scores = [0] * n_items

        for result in results:
            for position_idx,item_idx in enumerate(result.result):
                borda_scores[item_idx] += (n_items - (position_idx-1))

        indexed_scores = [(score, i) for i, score in enumerate(borda_scores)]    
        indexed_scores.sort(key=lambda x: (-x[0], x[1]))    
        final_ranking = [pos for _, pos in indexed_scores]

        if any(r.reason for r in results):
            reason = "Ensemble ranking based on multiple evaluations.\n" + '\n'.join([r.reason for r in results if r.reason])
        else:
            reason = None
        
        
        return MetricResult(result=result,reason=reason)

In [ ]:
# User instantiates a ranking metric by providing a name, an LLM, a prompt template, and the number of rankings desired.
my_ranking_metric = RankingMetrics(
    name='response_ranking',
    llm=LLM(),  # Your language model instance
    prompt="Rank the following responses:\n{candidates}",
    num_ranks=3,
)

# To score a single input (ranking candidate responses)
result = my_ranking_metric.score(candidates=[
    "short answer.",
    "a bit more detailed.",
    "the longest and most detailed answer."
],n=3)
print(result)   # Might output something like: [1, 0, 2]
print(result.reason)  # Provides the reasoning behind the ranking



/var/folders/ww/sk5dkfhn673234cmy5w7008r0000gn/T/ipykernel_95467/972172485.py:40: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  'result_validator': validator('result', allow_reuse=True)(validate_result)


[0, 1, 2]
Ensemble ranking based on multiple evaluations.
The responses are ranked from the shortest to the longest and most detailed.
The responses are ranked from shortest to longest and most detailed.
Responses ranked from shortest to longest.


### decorator factory for ranking metric

In [46]:
#| export
ranking_metric = create_metric_decorator(RankingMetrics)

In [48]:
@ranking_metric(
    llm=LLM(),  # Your language model instance
    prompt="Rank the following responses:\n{candidates}",
    name='new_ranking_metric',
    num_ranks=3
)
def my_ranking_metric(llm, prompt, **kwargs):
    # Your custom logic that calls the LLM and returns a tuple of (ranking, reason)
    # For example, process the prompt (formatted with candidates) and produce a ranking.
    ranking = [1, 0, 2]  # Dummy ranking: second candidate is best, then first, then third.
    reason = "Ranked based on response clarity and detail."
    return ranking, reason

# Using the decorator-based ranking metric:
result = my_ranking_metric.score(candidates=[
    "Response A: short answer.",
    "Response B: a bit more detailed.",
    "Response C: the longest and most detailed answer."
])
print(result)   # E.g., [1, 0, 2]
print(result.reason)  # E.g., "Ranked based on response clarity and detail."


[1, 0, 2]
Ranked based on response clarity and detail.
